In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import pickle

'''%load_ext autoreload
%autoreload 2'''

'%load_ext autoreload\n%autoreload 2'

In [3]:
# 刷新operators檔案
import importlib
import operators

importlib.reload(operators)
from operators import *


# 價量資料與報酬率處理

In [4]:
def read_tej_txt(file_path):
    with open(file_path, "rb") as f:
        raw = f.read()
    text = raw.decode("big5", errors="ignore")
    with open('clean'+file_path, "w", encoding="utf-8") as f:
        f.write(text)
    df = pd.read_table('clean'+file_path)
    return df

In [5]:
price_data = pd.read_table(r"./Input_data/raw_price.txt",encoding='ANSI')

In [5]:
# 價格資料處理
price_data = price_data[(price_data['市場別'] == 'TSE') | (price_data['市場別'] == 'OTC')]  # 過濾出上市櫃資料
price_data['證券代碼'] = price_data['證券代碼'].astype(str)
price_data[['證券代碼', 'firm']] = price_data['證券代碼'].str.split(' ', expand=True, n=1)
price_data.rename(columns={'證券代碼': 'ticker', '收盤價(元)': 'close', '開盤價(元)': 'open', '最高價(元)': 'high', 
                           '最低價(元)': 'low', '成交量(千股)': 'volume', '年月日':'date', '報酬率％': 'daily_ret',
                           '成交值(千元)': 'value_dollars', '市值(百萬元)': 'cap', '本益比-TSE': 'pe', '股價淨值比-TSE':'pb',
                            '股利殖利率-TSE': 'dividend_yield', '市場別': 'market'}, inplace=True)
price_data['date'] = pd.to_datetime(price_data['date'], format='%Y%m%d')
price_data['daily_ret'] = (price_data['close'] - price_data['open'])/ price_data['open']
price_data['vwap'] = price_data[['close', 'open', 'high', 'low']].mean(axis=1)
del price_data['firm']
price_data.sort_values(by=['ticker', 'date'], inplace=True)

for i in range(1,21):
    price_data[f'ret{i}'] = ((price_data.groupby('ticker')['close'].shift(-1-i) - price_data.groupby('ticker')['close'].shift(-1))/ price_data.groupby('ticker')['close'].shift(-1))  #  持有i日報酬率 (t1收買, ti+1收賣)
    #price_data[f'ret{i}'] = np.log(price_data.groupby('ticker')['close'].shift(-1-i)/ price_data.groupby('ticker')['close'].shift(-1))


In [56]:
問題值 = price_data[(price_data['ret1'] >= 0.2)]
問題值 = 問題值[['date', 'ticker', 'open', 'close', 'daily_ret' ,'ret1', 'ret2']]
問題值

,date,ticker,open,close,daily_ret,ret1,ret2
2179574,2024-06-28,2429,26.3774,26.5272,0.005679,0.466552,0.560892
3346114,2021-02-17,3073,12.4288,12.4288,0.000000,0.589550,0.746267
3387268,2010-05-12,3092,24.1591,21.4073,-0.113903,0.261261,0.255856
3583516,2017-11-28,3219,24.7895,24.7895,0.000000,0.207083,0.207083
3583518,2017-11-30,3219,32.8795,32.8795,0.000000,0.207677,0.241537
3584056,2020-02-19,3219,14.6899,14.7370,0.003206,0.207668,0.325880
3900309,2011-10-24,3379,24.7899,24.9492,0.006426,0.216219,0.299552
4354647,2010-04-27,3630,69.2372,83.9980,0.213192,0.310680,0.184466
4410421,2014-03-28,3684,19.4824,21.2015,0.088239,0.436363,0.290907
4581602,2006-06-13,4128,7.0122,6.8472,-0.023530,0.210191,0.127378


In [ ]:
# 合併資料
pv_data = price_data.copy()
pv_data.sort_values(by=['ticker', 'date'], inplace=True)
pv_data.set_index(['date', 'ticker'], inplace=True)

pv_data_dict = {col: pv_data[col].copy().unstack() for col in pv_data.columns}
# 把價量df寫出去
with open(r"./Input_data/pv_data_dict.pkl", "wb") as f:
    pickle.dump(pv_data_dict, f)

In [130]:
print(pv_data_dict.keys())

dict_keys(['open', 'high', 'low', 'close', 'volume', 'value_dollars', 'daily_ret', 'cap', 'pe', 'pb', 'dividend_yield', 'market', 'vwap', 'ret1', 'ret2', 'ret3', 'ret4', 'ret5', 'ret6', 'ret7', 'ret8', 'ret9', 'ret10', 'ret11', 'ret12', 'ret13', 'ret14', 'ret15', 'ret16', 'ret17', 'ret18', 'ret19', 'ret20'])


In [133]:
pd.to_pickle(price_data, r'./Input_data/price_data.pkl')

# 籌碼資料處理

In [13]:
flow_data = pd.read_table(r"./Input_data/raw_籌碼資料.txt",encoding='ANSI')
del flow_data['外資買賣超(張).1']

In [10]:
flow_data.columns

Index(['證券代碼', '年月日', '外資買賣超(張)', '投信買賣超(張)', '自營買賣超(張)', '融資餘額(張)',
       '融資餘額(千元)', '外資買超(張)', '外資賣超(張)', '投信買超(張)', '投信賣超(張)', '自營買超(張)',
       '自營賣超(張)', '外資總持股率_不含董監%', '投信持股率％', '自營持股率％', '董監持股％'],
      dtype='object')

In [14]:
flow_data['證券代碼'] = flow_data['證券代碼'].astype(str)
flow_data[['證券代碼', 'firm']] = flow_data['證券代碼'].str.split(' ', expand=True, n=1)
flow_data.rename(columns= {'證券代碼': 'ticker', '年月日': 'date', '外資買賣超(張)': 'foreign_net_buy_shares', '投信買賣超(張)': 'investment_trust_net_buy_shares',
                        '自營買賣超(張)': 'dealer_net_buy_shares',  '融資餘額(張)': 'margin_balance_shares', '融資餘額(千元)': 'margin_balance_dollars',
                        '外資總持股率_不含董監%': 'foreign_holding_ratio', '投信持股率％': 'investment_trust_holding_ratio' , '自營持股率％': 'dealer_holding_ratio',
                        '董監持股％': 'insider_holding_ratio'}, inplace=True)
flow_data['date'] = pd.to_datetime(flow_data['date'], format='%Y%m%d')
flow_data.sort_values(by=['ticker', 'date'], inplace= True)
del flow_data['firm']

In [16]:
flow_merged_df = price_data.merge(flow_data, on=['date', 'ticker'], how='left')
flow_merged_df = flow_merged_df[flow_merged_df['date'] >= '2014-01-01']
flow_col = ['foreign_net_buy_shares', 'investment_trust_net_buy_shares', 'dealer_net_buy_shares', 'margin_balance_shares',
            'margin_balance_dollars', 'foreign_holding_ratio', 'investment_trust_holding_ratio', 'dealer_holding_ratio', 'insider_holding_ratio']

In [ ]:
# 把df展開
flow_merged_df.set_index(['date', 'ticker'], inplace=True)
flow_data_dict = {col: flow_merged_df[col].copy().unstack() for col in flow_col}

with open(r"./Input_data/flow_data_dict.pkl", "wb") as f:
    pickle.dump(flow_data_dict, f)

# 財報資料處理
1. 損益表資料

In [24]:
is_data = pd.read_pickle(r"./Input_data/is_data_cleaned.pkl")
is_merged_df = price_data.merge(is_data, on=['date', 'ticker'], how='left')
is_merged_df = is_merged_df[is_merged_df['date'] >= '2014-01-01']
is_cols = ['net_sales', 'cogs', 'gross_margin', 'op_expense', 'op_income', 'eps', 'net_income', 'ebit', 'ebitda', 'ind_code']

In [25]:
# 把df展開
is_merged_df.set_index(['date', 'ticker'], inplace=True)
is_data_dict = {col: is_merged_df[col].copy().unstack() for col in is_cols}

2. 資產負債表資料

In [26]:
bs_data = pd.read_table(r"./Input_data/raw_bs_data.txt",encoding='ANSI')

# 項目整理
bs_data['證券代碼'] = bs_data['證券代碼'].astype(str)
bs_data[['證券代碼', 'firm']] = bs_data['證券代碼'].str.split(' ', expand=True, n=1)

bs_data.rename(columns={'證券代碼': 'ticker', '財報發布日': 'date', '  現金及約當現金': 'cash', '資產總額': 'total_asset', 
                           '負債總額': 'total_liability', '股東權益總額': 'total_equity', '流動資產':'current_asset', '非流動資產': 'non_current_asset',
                           '流動負債': 'current_liability', '非流動負債': 'non_current_liability', '  短期借款': 'st_borrowing', '  應收帳款及票據': 'ar',
                           '  存貨': 'inventory', '  應付帳款及票據': 'ap', '  不動產廠房及設備': 'ppe', '    商譽淨額': 'goodwill' }, inplace=True)

bs_data['ticker']=bs_data['ticker'].astype(str)
bs_data['date']=pd.to_datetime(bs_data['date'])
bs_data = bs_data[bs_data['date'].notna()]
bs_data = bs_data.sort_values(by=['ticker','date'])
del bs_data["firm"]
bs_data

,ticker,年月,date,cash,total_asset,total_liability,total_equity,current_asset,non_current_asset,current_liability,non_current_liability,st_borrowing,ar,inventory,ap,ppe,goodwill
0,1101,201003,2010-04-30,17568054,218351345,116193841,102157504,68129797,150221548,46716890,69476951,11434982,11268842,6679971,7581468,128527240,261375.0
1,1101,201006,2010-08-27,16624092,231153577,128804352,102349225,58415108,172738469,57452958,71351394,8996391,13587643,7867475,9443148,140355447,7561052.0
2,1101,201009,2010-11-05,12836138,231634057,122047752,109586305,59846089,171787968,47121934,74925818,7685856,14332045,7957196,8279490,139393786,7380777.0
3,1101,201012,2011-04-29,32706814,261960788,137447010,124513778,82001294,179959494,60121542,77325468,13170917,15918356,7296613,9650878,139708763,6875446.0
4,1101,201103,2011-05-03,21429405,259505224,130777073,128728151,78662336,180842888,55758562,75018511,14850145,17197382,8059799,10658778,140342510,6927578.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96275,9962,202409,2024-11-08,135684,1522104,423284,1098820,1093964,428140,411181,12103,340864,33247,882979,34196,410870,0.0
96276,9962,202412,2025-03-13,119459,1717651,610248,1107403,1279765,437886,600587,9661,548862,37451,1063907,4417,425274,0.0
96277,9962,202503,2025-05-12,136549,1575036,501337,1073699,1141772,433264,491770,9567,397451,14120,943274,19793,421519,0.0
96278,9962,202506,2025-08-12,158063,1523103,474722,1048381,1089746,433357,465529,9193,399597,77555,823206,35216,417676,0.0


In [27]:
# 清理同日期複數發布 (只保留最新的)
duplicates = bs_data[bs_data.duplicated(subset=['date', 'ticker'], keep=False)]
keep_max = duplicates.sort_values('年月', ascending=False).drop_duplicates(subset=['date', 'ticker'], keep='first')
non_duplicates = bs_data.drop(duplicates.index)
cleaned_data = pd.concat([non_duplicates, keep_max], ignore_index=True)
bs_data = cleaned_data.sort_values(['ticker', 'date']).reset_index(drop=True)

In [30]:
bs_merged_df = price_data.merge(bs_data, on=['date', 'ticker'], how='left')
ffill_col=[col for col in bs_data.columns.tolist() if col not in ['ticker', 'date']]

bs_merged_df[ffill_col] = (bs_merged_df.groupby('ticker')[ffill_col].ffill().reset_index(drop=True))
bs_merged_df = bs_merged_df[bs_merged_df['date'] >= '2014-01-01']
bs_cols = ['cash', 'total_asset', 'total_liability',
       'total_equity', 'current_asset', 'non_current_asset',
       'current_liability', 'non_current_liability', 'st_borrowing', 'ar',
       'inventory', 'ap', 'ppe', 'goodwill']

In [31]:
# 把df展開
bs_merged_df.set_index(['date', 'ticker'], inplace=True)
bs_data_dict = {col: bs_merged_df[col].copy().unstack() for col in bs_cols}

3. 現金流量表資料

In [32]:
cfs_data = pd.read_table(r"./Input_data/raw_cfs_data.txt",encoding='ANSI')

# 項目整理
cfs_data['證券代碼'] = cfs_data['證券代碼'].astype(str)
cfs_data[['證券代碼', 'firm']] = cfs_data['證券代碼'].str.split(' ', expand=True, n=1)

cfs_data.rename(columns={'證券代碼': 'ticker', '財報發布日': 'date', '來自營運之現金流量': 'cfo', '投資活動之現金流量': 'cfi', 
                           '籌資活動之現金流量': 'cff', '期初現金及約當現金': 'beg_cash', '期末現金及約當現金':'end_cash', '自由現金流量(D)': 'fcf',
                           '  購置不動產廠房設備（含預付）－CFI': 'capex'}, inplace=True)

cfs_data['ticker']=cfs_data['ticker'].astype(str)
cfs_data['date']=pd.to_datetime(cfs_data['date'])
cfs_data = cfs_data[cfs_data['date'].notna()]
cfs_data = cfs_data.sort_values(by=['ticker','date'])
del cfs_data["firm"]
cfs_data

,ticker,年月,date,cfo,cfi,cff,beg_cash,end_cash,fcf,capex
0,1101,201003,2010-04-30,4106544.0,-13840210.0,10508301.0,16426107.0,17568054,-16411559.0,-1705409.0
1206,1101,201006,2010-08-27,-460324.0,1255236.0,-1595374.0,17568054.0,16624092,24087003.0,-1482220.0
2542,1101,201009,2010-11-05,4143204.0,-4209678.0,-2768811.0,16624092.0,12836138,-8609557.0,-2947645.0
3763,1101,201012,2011-04-29,6025330.0,-14926034.0,27671803.0,12836138.0,32706814,-7902019.0,-5370254.0
5101,1101,201103,2011-05-03,3061775.0,-2992504.0,-11195373.0,32706814.0,21429405,521497.0,-2697365.0
...,...,...,...,...,...,...,...,...,...,...
89353,9962,202409,2024-11-08,120857.0,-23579.0,-83074.0,121480.0,135684,-7272.0,-23149.0
91084,9962,202412,2025-03-13,-211028.0,-13100.0,207903.0,135684.0,119459,7358.0,-13101.0
92815,9962,202503,2025-05-12,174943.0,-11293.0,-146560.0,119459.0,136549,37927.0,-10858.0
94547,9962,202506,2025-08-12,67759.0,-3132.0,-43113.0,136549.0,158063,6185.0,-3132.0


In [33]:
# 清理同日期複數發布 (只保留最新的)
duplicates = cfs_data[cfs_data.duplicated(subset=['date', 'ticker'], keep=False)]
keep_max = duplicates.sort_values('年月', ascending=False).drop_duplicates(subset=['date', 'ticker'], keep='first')
non_duplicates = cfs_data.drop(duplicates.index)
cleaned_data = pd.concat([non_duplicates, keep_max], ignore_index=True)
cfs_data = cleaned_data.sort_values(['ticker', 'date']).reset_index(drop=True)

In [34]:
cfs_merged_df = price_data.merge(cfs_data, on=['date', 'ticker'], how='left')
ffill_col=[col for col in cfs_data.columns.tolist() if col not in ['ticker', 'date']]

cfs_merged_df[ffill_col] = (cfs_merged_df.groupby('ticker')[ffill_col].ffill().reset_index(drop=True))
cfs_merged_df = cfs_merged_df[cfs_merged_df['date'] >= '2014-01-01']
cfs_cols = ['cfo', 'cfi', 'cff', 'beg_cash', 'end_cash',
       'fcf', 'capex']

In [35]:
# 把df展開
cfs_merged_df.set_index(['date', 'ticker'], inplace=True)
cfs_data_dict = {col: cfs_merged_df[col].copy().unstack() for col in cfs_cols}

4. 月營收資料

In [36]:
sales_data = pd.read_pickle('sales_df.pkl')
sales_data = sales_data[sales_data['date'] >= '2014-01-01']
sales_data.set_index(['date', 'ticker'], inplace=True)
sales_data.rename(columns = {'單月營收(千元)': 'monthly_sales', 'sales_MOM': 'sales_mom', 'sales_YOY': 'sales_yoy'}, inplace= True)

sales_col = ['monthly_sales', 'sales_mom', 'sales_yoy']
sales_data_dict = {col: sales_data[col].copy().unstack() for col in sales_col}

5. 整合三大報表資訊並輸出

In [37]:
fs_data_dict = is_data_dict | bs_data_dict | cfs_data_dict | sales_data_dict
print(fs_data_dict.keys())

# 把財報df寫出去
with open(r"./Input_data/fs_data_dict.pkl", "wb") as f:
    pickle.dump(fs_data_dict, f)

dict_keys(['net_sales', 'cogs', 'gross_margin', 'op_expense', 'op_income', 'eps', 'net_income', 'ebit', 'ebitda', 'ind_code', 'cash', 'total_asset', 'total_liability', 'total_equity', 'current_asset', 'non_current_asset', 'current_liability', 'non_current_liability', 'st_borrowing', 'ar', 'inventory', 'ap', 'ppe', 'goodwill', 'cfo', 'cfi', 'cff', 'beg_cash', 'end_cash', 'fcf', 'capex', 'monthly_sales', 'sales_mom', 'sales_yoy'])


# 情緒因子

In [16]:
factor_data = pd.read_pickle(r"./Input_data/sentiment_factor.pkl")
factor_data.iloc[99528]

證券代碼              2353 宏焎SEP]20140103
年月日                                 1
則次          Chromebook殺出重圍  NB廠趁勢再燃戰火
factor_1                            1
factor_2                            1
factor                            1.0
Name: 99529, dtype: object

In [17]:
target_len = 8

mask = factor_data["年月日"].astype(str).str.len() == target_len
factor_data = factor_data.loc[mask].copy()

In [18]:
factor_data['證券代碼'] = factor_data['證券代碼'].astype(str)
factor_data[['證券代碼', 'firm']] = factor_data['證券代碼'].str.split(' ', expand=True, n=1)
factor_data.rename(columns={'證券代碼': 'ticker', '年月日':'date'}, inplace=True)
factor_data['date'] = pd.to_datetime(factor_data['date'], format='%Y%m%d')
del factor_data['firm']
factor_data.sort_values(by=['ticker', 'date'], inplace=True)
factor_data

,ticker,date,則次,factor_1,factor_2,factor
1,1101,2014-01-10,1,1,-1,0.0
2,1101,2014-01-11,1,0,0,0.0
3,1101,2014-01-13,1,0,0,0.0
4,1101,2014-01-21,1,0,0,0.0
5,1101,2014-01-28,1,0,0,0.0
...,...,...,...,...,...,...
462296,9962,2025-03-14,2,0,-1,-0.5
462297,9962,2025-03-18,1,0,0,0.0
462298,9962,2025-04-17,1,0,0,0.0
462299,9962,2025-05-07,1,0,1,0.5


In [21]:
# 把同天有複數則新聞的因子值累加
factor_agg = (
    factor_data
    .groupby(["ticker", "date"], as_index=False)
    .agg({"factor": "sum"})
)
factor_agg

,ticker,date,factor
0,1101,2014-01-10,0.0
1,1101,2014-01-11,0.0
2,1101,2014-01-13,0.0
3,1101,2014-01-21,0.0
4,1101,2014-01-28,0.0
...,...,...,...
381513,9962,2025-03-14,-1.5
381514,9962,2025-03-18,0.0
381515,9962,2025-04-17,0.0
381516,9962,2025-05-07,0.5


In [ ]:
factor_agg.set_index(['date', 'ticker'], inplace=True)
factor_df = factor_agg['factor'].unstack()

factor                                               ...            \
ticker       1101 1102 1103 1104 1108 1109 1110 1201 1203 1210  ... 9944 9945   
date                                                            ...             
2014-01-01    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2014-01-02    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2014-01-03    NaN  0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2014-01-04    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2014-01-05    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
...           ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2025-06-25    NaN  NaN  NaN  0.0  NaN  NaN  NaN  NaN  NaN  NaN  ...  0.0  NaN   
2025-06-26    0.5  NaN  0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2025-06-27    NaN  NaN  NaN  NaN  0.0  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2025-06-28    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   
2025-06-30    0.0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...  NaN  NaN   

                                                    
ticker     9946 9949 9950 9951 9955 9958 9960 9962  
date                                                
2014-01-01  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2014-01-02  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2014-01-03  NaN  NaN  0.0  NaN  0.0  NaN  NaN  NaN  
2014-01-04  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2014-01-05  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
...         ...  ...  ...  ...  ...  ...  ...  ...  
2025-06-25  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2025-06-26  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2025-06-27  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2025-06-28  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2025-06-30  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[4002 rows x 1752 columns]